In [1]:
import os
import pandas as pd
import numpy as np
import openai
from datetime import datetime
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import PGVector
from dotenv import load_dotenv
# from langchain_community.retrievers import MergerRetriever  

# 변경 부분 - Google genai, and PGVector
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_postgres.vectorstores import PGVector
from langchain_community.document_loaders import DirectoryLoader, TextLoader
import time
from dotenv import load_dotenv


#Openai 추가
from langchain_openai import OpenAIEmbeddings
from langchain_postgres import PGVector
from langchain_core.documents import Document
from tqdm import tqdm
import time

In [2]:
# .env 파일 로드 (코드 최상단에 위치해야 함!)

# API KEY 체크
load_dotenv(override=True)

# API 키 확인
api_key = os.getenv('OPENAI_API_KEY')
print(f"API 키: {api_key[:8]}")
CONNECTION_STRING = os.getenv("DATABASE_URL")
print(f"DB connection string : {CONNECTION_STRING[:3]}")

# 만약 None이면 문제!
if api_key is None:
    print("❌ .env 파일에 OPENAI_API_KEY가 없습니다!")

if CONNECTION_STRING is None:
    print("❌ .env 파일에 CONNECTION_STRING가 없습니다!")
    
# OpenAI 임베딩 객체 생성
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",  # 또는 "text-embedding-3-large", "text-embedding-ada-002"
    openai_api_key=api_key  # 명시적으로 전달 (선택사항)
)

print("✅ OpenAI 임베딩 모델 로드 완료")


API 키: sk-proj-
DB connection string : pos
✅ OpenAI 임베딩 모델 로드 완료


In [8]:
# 데이터 로드
df = pd.read_excel("SI_data/2025전체.xlsx")

print(f"총 {len(df)}개 행")
print(f"컬럼: {df.columns.tolist()}")

총 70086개 행
컬럼: ['No', '접수일자', '접수자', '가맹분류', '가맹점명', '상담요약', '접수내용', '처리결과', '처리일시', '처리자1', '처리자2', '처리자3', '처리유형', '대분류', '중분류', '출장수리내역', '모바일']


In [9]:
def preprocess_dataframe(df):
    """데이터 정제"""
    # 결측치 처리
    df['상담요약'] = df['상담요약'].fillna('')
    df['접수내용'] = df['접수내용'].fillna('')
    df['처리결과'] = df['처리결과'].fillna('처리중')
    df['가맹분류'] = df['가맹분류'].fillna('미분류')
    df['대분류'] = df['대분류'].fillna('기타')
    df['중분류'] = df['중분류'].fillna('기타')
    
    # 날짜 형식 통일
    df['접수일자'] = pd.to_datetime(df['접수일자'], errors='coerce')
    df['처리일시'] = pd.to_datetime(df['처리일시'], errors='coerce')
    
    # 공백 정리
    text_columns = ['상담요약', '접수내용', '처리결과']
    for col in text_columns:
        df[col] = df[col].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()
    
    return df

df = preprocess_dataframe(df)
print("✅ 데이터 정제 완료")

✅ 데이터 정제 완료


In [10]:
def row_to_text(row):
    """각 행을 검색 가능한 텍스트로 변환"""
    text = f"""
[접수번호 {row['No']}]
접수일시: {row['접수일자'].strftime('%Y-%m-%d %H:%M') if pd.notna(row['접수일자']) else '미상'}
가맹점: {row['가맹분류']} - {row['가맹점명']}

문제 요약: {row['상담요약']}

상세 내용:
{row['접수내용']}

처리 결과:
{row['처리결과']}

분류: {row['대분류']} > {row['중분류']}
처리 방식: {row['처리유형']}
담당자: {row['처리자1']}
    """.strip()
    return text

def create_metadata(row):
    """메타데이터 생성"""
    return {
        'no': int(row['No']),
        'date': row['접수일자'].strftime('%Y-%m-%d') if pd.notna(row['접수일자']) else None,
        'category': str(row['가맹분류']),
        'store_name': str(row['가맹점명']),
        'summary': str(row['상담요약']),
        'type': str(row['처리유형']),
        'main_category': str(row['대분류']),
        'sub_category': str(row['중분류']),
        'year': int(row['접수일자'].year) if pd.notna(row['접수일자']) else None,
        'month': int(row['접수일자'].month) if pd.notna(row['접수일자']) else None
    }

# 테스트
print("📝 텍스트 변환 예시:")
print(row_to_text(df.iloc[0]))

📝 텍스트 변환 예시:
[접수번호 1]
접수일시: 2025-01-01 07:10
가맹점: 투썸플레이스 - 투썸_신탄진휴게소(상)점

문제 요약: DID 오류

상세 내용:
ODS DID 호출 안됨

처리 결과:
장치관리자 com9 제거 후 재부팅. 이후 정상

분류: 장비 > DID
처리 방식: 원격
담당자: 고정현


In [11]:
def create_chunks(df):
    """전체 데이터를 청크로 변환"""
    chunks = []
    
    for idx, row in df.iterrows():
        chunk = {
            'text': row_to_text(row),
            'metadata': create_metadata(row)
        }
        chunks.append(chunk)
    
    return chunks

chunks = create_chunks(df)
print(f"✅ 총 {len(chunks)}개 청크 생성 완료")

# 청크 확인
print("\n첫 번째 청크:")
print(chunks[0]['text'])
print("\n메타데이터:")
print(chunks[0]['metadata'])

✅ 총 70086개 청크 생성 완료

첫 번째 청크:
[접수번호 1]
접수일시: 2025-01-01 07:10
가맹점: 투썸플레이스 - 투썸_신탄진휴게소(상)점

문제 요약: DID 오류

상세 내용:
ODS DID 호출 안됨

처리 결과:
장치관리자 com9 제거 후 재부팅. 이후 정상

분류: 장비 > DID
처리 방식: 원격
담당자: 고정현

메타데이터:
{'no': 1, 'date': '2025-01-01', 'category': '투썸플레이스', 'store_name': '투썸_신탄진휴게소(상)점', 'summary': 'DID 오류', 'type': '원격', 'main_category': '장비', 'sub_category': 'DID', 'year': 2025, 'month': 1}


In [16]:
# PGVector 저장소 생성
vectorstore = PGVector(
    embeddings=embeddings,
    collection_name="SI_try",
    connection=CONNECTION_STRING,
    use_jsonb=True
)

print("✅ PGVector 연결 완료")

✅ PGVector 연결 완료


In [24]:
documents = []
for chunk in chunks:
    doc = Document(
        page_content=chunk['text'],
        metadata=chunk['metadata']
    )
    documents.append(doc)

print(f"✅ 총 {len(documents)}개 문서 생성")

# 배치 크기 설정
BATCH_SIZE = 500  # 한 번에 500개씩
total_docs = len(documents)
total_batches = (total_docs // BATCH_SIZE) + 1

print(f"📦 총 {total_docs:,}개 문서를 {BATCH_SIZE}개씩 {total_batches}배치로 저장")
print(f"⏱️ 예상 소요 시간: 약 {total_batches * 2}~{total_batches * 4}분")

start_time = time.time()

for i in tqdm(range(0, total_docs, BATCH_SIZE), desc="저장 중"):
    batch = documents[i:i+BATCH_SIZE]
    
    try:
        if i == 0:
            # 첫 배치: 컬렉션 생성
            vectorstore = PGVector.from_documents(
                documents=batch,
                embedding=embeddings,
                collection_name="SI_try",
                connection=CONNECTION_STRING,
                use_jsonb=True,
                pre_delete_collection=True  # 기존 데이터 삭제
            )
            print(f"✅ 컬렉션 생성 및 첫 {len(batch)}개 저장")
        else:
            # 이후 배치: 추가
            vectorstore.add_documents(batch)
        
        # 진행상황 출력
        processed = min(i + BATCH_SIZE, total_docs)
        elapsed = time.time() - start_time
        progress = processed / total_docs
        eta = (elapsed / progress) - elapsed if progress > 0 else 0
        
        print(f"📊 진행: {processed:,}/{total_docs:,} ({progress*100:.1f}%) | "
              f"소요: {elapsed/60:.1f}분 | 남은시간: {eta/60:.1f}분")
        
    except Exception as e:
        print(f"❌ 배치 {i//BATCH_SIZE + 1} 저장 실패: {e}")
        break

total_time = time.time() - start_time
print(f"\n🎉 완료! 총 소요시간: {total_time/60:.1f}분")

✅ 총 70086개 문서 생성
📦 총 70,086개 문서를 500개씩 141배치로 저장
⏱️ 예상 소요 시간: 약 282~564분


저장 중:   1%|▌                                                                        | 1/141 [00:05<11:50,  5.07s/it]

✅ 컬렉션 생성 및 첫 500개 저장
📊 진행: 500/70,086 (0.7%) | 소요: 0.1분 | 남은시간: 11.8분


저장 중:   1%|█                                                                        | 2/141 [00:09<10:53,  4.70s/it]

📊 진행: 1,000/70,086 (1.4%) | 소요: 0.2분 | 남은시간: 11.0분


...

📊 진행: 69,500/70,086 (99.2%) | 소요: 9.9분 | 남은시간: 0.1분


저장 중:  99%|██████████████████████████████████████████████████████████████████████▍| 140/141 [10:00<00:04,  4.23s/it]

📊 진행: 70,000/70,086 (99.9%) | 소요: 10.0분 | 남은시간: 0.0분


저장 중: 100%|███████████████████████████████████████████████████████████████████████| 141/141 [10:01<00:00,  4.27s/it]

📊 진행: 70,086/70,086 (100.0%) | 소요: 10.0분 | 남은시간: 0.0분

🎉 완료! 총 소요시간: 10.0분


In [18]:
def search_tickets(query, category_filter=None, k=5):
    """이슈 검색"""
    
    if category_filter:
        # 메타데이터 필터링
        results = vectorstore.similarity_search(
            query,
            k=k,
            filter={'category': category_filter}
        )
    else:
        results = vectorstore.similarity_search(query, k=k)
    
    return results

# 사용 예시
results = SI_try(
    query="투썸플레이스 DID 오류",
    category_filter="투썸플레이스",
    k=3
)

for i, doc in enumerate(results, 1):
    print(f"\n[결과 {i}]")
    print(doc.page_content)
    print(f"메타데이터: {doc.metadata}")

NameError: name 'SI_try' is not defined

In [13]:
split_documents=chunks

In [14]:
# for doc in vectorstore.similarity_search("투썸플레이스"):
#     print(doc.page_content)

In [13]:
#chunk 900개만 해볼게요.
batch_size = 80
split_documents_900 = split_documents[:900]  # ← 앞에서 300개만
total_docs = len(split_documents_900)


for i in range(0, total_docs, batch_size):
    batch = split_documents_900[i:i+batch_size]
    current = min(i + batch_size, total_docs)
    progress = current / total_docs * 100
    batch_num = i // batch_size + 1
    
    print(f"[{progress:5.1f}%] 배치 {batch_num} | {i:3d}~{current-1:3d} ({len(batch):2d}개)", end=" ")
    
    try:
        if vectorstore is None:
            vectorstore = PGVector.from_documents(
                documents=batch,
                embedding=embeddings,
                connection=CONNECTION_STRING,
                collection_name="as_legacy_try"
            )
        else:
            vectorstore.add_documents(batch)
        
        print("✅")
        
        # 대기
        if current < total_docs:
            print(f"          ⏳ 60초 대기...", end="", flush=True)
            time.sleep(60)
            print(" 완료")
        
    except Exception as e:
        if "RESOURCE_EXHAUSTED" in str(e):
            print("❌ 할당량 초과")
            print(f"          ⏳ 80초 대기 후 재시도...", end="", flush=True)
            time.sleep(80)
            
            if vectorstore is None:
                vectorstore = PGVector.from_documents(
                    documents=batch,
                    embedding=embeddings,
                    connection=CONNECTION_STRING,
                    collection_name="as_legacy_try"
                )
            else:
                vectorstore.add_documents(batch)
            print(" ✅")
        else:
            print(f"\n❌ 에러: {e}")
            raise e

print(f"\n🎉 완료! {total_docs}개 문서가 'asp_try' 컬렉션에 저장되었습니다.")

[  8.9%] 배치 1 |   0~ 79 (80개) ✅
          ⏳ 60초 대기... 완료
[ 17.8%] 배치 2 |  80~159 (80개) ✅
          ⏳ 60초 대기... 완료
[ 26.7%] 배치 3 | 160~239 (80개) ✅
          ⏳ 60초 대기... 완료
[ 35.6%] 배치 4 | 240~319 (80개) ✅
          ⏳ 60초 대기... 완료
[ 44.4%] 배치 5 | 320~399 (80개) ✅
          ⏳ 60초 대기... 완료
[ 53.3%] 배치 6 | 400~479 (80개) ✅
          ⏳ 60초 대기... 완료
[ 62.2%] 배치 7 | 480~559 (80개) ✅
          ⏳ 60초 대기... 완료
[ 71.1%] 배치 8 | 560~639 (80개) ✅
          ⏳ 60초 대기... 완료
[ 80.0%] 배치 9 | 640~719 (80개) ✅
          ⏳ 60초 대기... 완료
[ 88.9%] 배치 10 | 720~799 (80개) ✅
          ⏳ 60초 대기... 완료
[ 97.8%] 배치 11 | 800~879 (80개) ✅
          ⏳ 60초 대기... 완료
[100.0%] 배치 12 | 880~899 (20개) ✅

🎉 완료! 900개 문서가 'asp_try' 컬렉션에 저장되었습니다.


In [14]:
for doc in vectorstore.similarity_search("화면꺼짐"):
    print(doc.page_content)

7.조치 :
*전면 - 유리만 교체 가능한지 패널 전체 교체해야 하는지 확인및 점검요청 
*후면 -HDMI 및 터치케이블 재연결해도 동일하여 점검 요청
■ 코너 : 가판대05_키오스크
■ 비용 : 유
■ 접수자 : 01097735543
■ 증상 : 화면 안 나옴. 터치 안 됨. 장비 전원은 들어온다고 함. HDMI 케이블 잘 모르셔서 포트 전체 꾹꾹 눌러보시도록 안내드렸으나 동일. 원격도 비활성화 된 상태라 정확한 확인 어려움.  
010-4141-5236(최초 접수자, 금일 휴무)
*방문시 신분증 지참요청|DID 부팅 불량.|SSD재연결 및 전원아답터 점검, 자동종료 셋팅함
7. 조치 : 이물질 제거 및 장비 재부팅, 리셋 해보았으나 동일 
8. 시간 : 6/30 오전 
9. 출장비용및현장납부안내 : 드림


In [16]:
retriever = vectorstore.as_retriever()
retriever.invoke("화면꺼짐")

[Document(id='04bbc31a-fd61-4592-942d-28ef365e4168', metadata={'source': 'as_data\\MTCDAYTB_202602021618.md'}, page_content='7.조치 :\n*전면 - 유리만 교체 가능한지 패널 전체 교체해야 하는지 확인및 점검요청 \n*후면 -HDMI 및 터치케이블 재연결해도 동일하여 점검 요청'),
 Document(id='f058b079-2a36-4bd8-a398-2b525504c81e', metadata={'source': 'as_data\\MTCDAYTB_202602021618.md'}, page_content='■ 코너 : 가판대05_키오스크\n■ 비용 : 유\n■ 접수자 : 01097735543\n■ 증상 : 화면 안 나옴. 터치 안 됨. 장비 전원은 들어온다고 함. HDMI 케이블 잘 모르셔서 포트 전체 꾹꾹 눌러보시도록 안내드렸으나 동일. 원격도 비활성화 된 상태라 정확한 확인 어려움.  \n010-4141-5236(최초 접수자, 금일 휴무)'),
 Document(id='f5b82a9d-87a1-4d02-a8aa-6aab07e2b177', metadata={'source': 'as_data\\MTCDAYTB_202602021618.md'}, page_content='*방문시 신분증 지참요청|DID 부팅 불량.|SSD재연결 및 전원아답터 점검, 자동종료 셋팅함'),
 Document(id='b8ac5b90-ef73-4bff-8979-22bbde7ef08a', metadata={'source': 'as_data\\MTCDAYTB_202602021618.md'}, page_content='7. 조치 : 이물질 제거 및 장비 재부팅, 리셋 해보았으나 동일 \n8. 시간 : 6/30 오전 \n9. 출장비용및현장납부안내 : 드림')]

In [17]:
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

In [18]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [21]:
question = "화면 꺼짐 발생시 어떻게 as 처리했어?"
response = chain.invoke(question)
print(response)

화면 꺼짐 발생 시 AS 처리 내역은 다음과 같습니다:

1.  **조치 요청:**
    *   전면의 경우 유리만 교체 가능한지 또는 패널 전체를 교체해야 하는지 확인 및 점검을 요청했습니다.
    *   후면의 경우 HDMI 및 터치 케이블을 재연결해도 동일한 문제가 발생하여 점검을 요청했습니다.
2.  **처리 내역 (수리):**
    *   메인보드 수리 및 LVDS 케이블 교체가 유상으로 진행되었으며 (204,000원 VAT 별도), AS 장비 수리 완료 후 매장 장비 설치 대체 및 회수가 완료되었습니다.
    *   다른 처리 내역으로는 메인보드 수리, LVDS 케이블 및 메카 교체가 유상으로 진행되었으며 (302,000원 VAT 별도), AS 장비 수리 완료 후 7월 21일 재설치가 완료되었습니다.


In [1]:
# # 현재 노트북을 .py로 변환
# !jupyter nbconvert --to python SI_AS_Embedding.ipynb --output ./python_scripts/SI_AS_Embedding.py

[NbConvertApp] Converting notebook SI_AS_Embedding.ipynb to python
[NbConvertApp] Writing 9106 bytes to python_scripts\SI_AS_Embedding.py
